In [3]:
#add 3 new, features to our district-level ML dataset from our LSTM model
from tensorflow.keras.models import load_model
import joblib
import numpy as np
import pandas as pd

model_ms = load_model(r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\models\global_multistep_lstm.keras")

scaler_multi = joblib.load("C:/Ibrahim/Personal/University Stuff/Machine Learning/Project/ML Irrigation Project/models/lstm_scaler.pkl")

print("lstm model & scaler loaded.")

lstm model & scaler loaded.


In [4]:
rain_path = r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\raw\Rain_fall_in_Pakistan.csv"

rain = pd.read_csv(rain_path)
rain = rain.iloc[1:]  #remove metadata row

rain["date"] = pd.to_datetime(rain["date"], errors="coerce")

#REQUIRED FEATURES FOR LSTM
FEATURES = ['rfh', 'rfh_avg', 'r1h', 'r3h', 'n_pixels']

#convert to numeric
for col in FEATURES:
    rain[col] = pd.to_numeric(rain[col], errors="coerce")

#fill missing exactly like LSTM training
rain[FEATURES] = rain[FEATURES].bfill().ffill()

#keeping ALL 5 features + metadata
rain_small = rain[['ADM2_PCODE', 'date'] + FEATURES].copy()

rain_small["Year"] = rain_small["date"].dt.year

#sort ascending
rain_small = rain_small.sort_values(['ADM2_PCODE', 'date']).reset_index(drop=True)

rain_small.head()


,ADM2_PCODE,date,rfh,rfh_avg,r1h,r3h,n_pixels,Year
0,PK203,1981-01-01,9.0142,7.8681,40.9483,95.7615,2398.0,1981
1,PK203,1981-01-11,13.4554,10.0483,40.9483,95.7615,2398.0,1981
2,PK203,1981-01-21,18.4787,10.3431,40.9483,95.7615,2398.0,1981
3,PK203,1981-02-01,11.6697,8.8762,43.6038,95.7615,2398.0,1981
4,PK203,1981-02-11,7.8745,9.0815,38.0229,95.7615,2398.0,1981


In [5]:
df = pd.read_csv(r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\processed\final_cleaned_district_dataset.csv")

df.head()

,District,Year,Avg_Rainfall,Avg_Temperature,Crop_Yield,Irrigation_Area
0,PK203,1981,3.423858,20.206408,2973.113131,118.704918
1,PK203,1982,6.698594,19.510198,2973.113131,118.704918
2,PK203,1983,3.331681,19.611718,2973.113131,118.704918
3,PK203,1984,2.419928,19.920437,2973.113131,118.704918
4,PK203,1985,2.637431,20.242908,2973.113131,118.704918


In [6]:
#inverse-scale rainfall predictions
def inverse_scale_prediction(pred_scaled, scaler):
    pred_scaled = np.array(pred_scaled).reshape(-1,1)

    #create dummy row with 5 features bec scaler expects 5 columns
    dummy = np.hstack([pred_scaled, np.zeros((len(pred_scaled),4))])

    inv = scaler.inverse_transform(dummy)

    return inv[:,0]  #rainfall is first col


In [9]:
rain_path = r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\raw\Rain_fall_in_Pakistan.csv"

rain = pd.read_csv(rain_path)
rain = rain.iloc[1:]  # remove metadata row

rain['date'] = pd.to_datetime(rain['date'], errors='coerce')

FEATURES = ['rfh', 'rfh_avg', 'r1h', 'r3h', 'n_pixels']

# Convert to numeric and clean missing the same way we trained LSTM
for col in FEATURES:
    rain[col] = pd.to_numeric(rain[col], errors='coerce')

rain[FEATURES] = rain[FEATURES].bfill().ffill()

rain_small = rain[['ADM2_PCODE', 'date'] + FEATURES].copy()
rain_small['Year'] = rain_small['date'].dt.year

rain_small = rain_small.sort_values(['ADM2_PCODE', 'date']).reset_index(drop=True)

rain_small.head()

,ADM2_PCODE,date,rfh,rfh_avg,r1h,r3h,n_pixels,Year
0,PK203,1981-01-01,9.0142,7.8681,40.9483,95.7615,2398.0,1981
1,PK203,1981-01-11,13.4554,10.0483,40.9483,95.7615,2398.0,1981
2,PK203,1981-01-21,18.4787,10.3431,40.9483,95.7615,2398.0,1981
3,PK203,1981-02-01,11.6697,8.8762,43.6038,95.7615,2398.0,1981
4,PK203,1981-02-11,7.8745,9.0815,38.0229,95.7615,2398.0,1981


In [13]:
from tensorflow.keras.models import load_model
import numpy as np

SEQ_LEN = 60
FUTURE_STEPS = 24  # Your LSTM output size = 24 hours ahead

model_ms = load_model(
    r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\models\global_multistep_lstm.keras"
)


import joblib
scaler_multi = joblib.load("C:/Ibrahim/Personal/University Stuff/Machine Learning/Project/ML Irrigation Project/models/lstm_scaler.pkl")


def inverse_scale_prediction(pred_scaled, scaler):
    dummy = np.zeros((len(pred_scaled), 5))          #5 features
    dummy[:, 0] = pred_scaled                        #rainfall placed in col 0
    inv = scaler.inverse_transform(dummy)
    return inv[:, 0]                                 #return rainfall only


#forecast function
def forecast_for_year(rdist):
    """
    rdist = rainfall rows *before* the target year
    returns (next_1h, next_24h_avg, next_7day_avg)
    """
    if len(rdist) < SEQ_LEN:
        return np.nan, np.nan, np.nan

    #build last-60-sequence across all 5 features
    seq = rdist[FEATURES].tail(SEQ_LEN).values.reshape(1, SEQ_LEN, len(FEATURES))

    # 1 model run → 24-hr forecast (scaled)
    pred24_scaled = model_ms.predict(seq, verbose=0)[0]

    # inverse-scale
    pred24 = inverse_scale_prediction(pred24_scaled, scaler_multi)

    next_1  = pred24[0]
    next_24 = pred24.mean()

    # fast synthetic 7-day forecast (no 168-loop)
    next_7  = pred24.mean() * 1.05    # small inflation factor (optional)

    return next_1, next_24, next_7


In [14]:
from tqdm import tqdm

df["Rain_Next_1"] = np.nan
df["Rain_Next_24"] = np.nan
df["Rain_Next_7d"] = np.nan

districts = df["District"].unique()

print("\nStarting forecasting...\n")

for district in tqdm(districts, desc="Districts"):
    
    # rainfall history for this district
    rdist_all = rain_small[rain_small["ADM2_PCODE"] == district].copy()
    if len(rdist_all) < SEQ_LEN:
        continue

    rdist_all = rdist_all.sort_values("date")

    # unique years that exist in ML dataset
    years = df[df["District"] == district]["Year"].unique()

    for year in years:
        # rainfall BEFORE this year
        rdist = rdist_all[rdist_all["Year"] < year]

        if len(rdist) < SEQ_LEN:
            continue

        # get predictions
        n1, n24, n7 = forecast_for_year(rdist)

        # write back into dataset
        df.loc[
            (df["District"] == district) & (df["Year"] == year),
            ["Rain_Next_1", "Rain_Next_24", "Rain_Next_7d"]
        ] = [n1, n24, n7]

print("\nForecast merging complete!")


Starting forecasting...



Districts: 100%|██████████| 30/30 [01:09<00:00,  2.32s/it]


Forecast merging complete!


In [15]:
df.head(10)

,District,Year,Avg_Rainfall,Avg_Temperature,Crop_Yield,Irrigation_Area,Rain_Next_1,Rain_Next_24,Rain_Next_7d
0,PK203,1981,3.423858,20.206408,2973.113131,118.704918,NaN,NaN,NaN
1,PK203,1982,6.698594,19.510198,2973.113131,118.704918,NaN,NaN,NaN
2,PK203,1983,3.331681,19.611718,2973.113131,118.704918,10.591415,2.269813,2.383304
3,PK203,1984,2.419928,19.920437,2973.113131,118.704918,11.673900,2.423514,2.544689
4,PK203,1985,2.637431,20.242908,2973.113131,118.704918,11.465340,2.390116,2.509621
5,PK203,1986,2.657394,19.578992,2973.113131,118.704918,11.417544,2.382497,2.501622
6,PK203,1987,2.810428,20.427150,2973.113131,118.704918,11.513817,2.397244,2.517106
7,PK203,1988,3.055322,20.858067,2973.113131,118.704918,11.346742,2.370924,2.489470
8,PK203,1989,3.025525,19.541077,2973.113131,118.704918,11.504424,2.396040,2.515842
9,PK203,1990,3.990772,20.353342,2973.113131,118.704918,11.451955,2.389119,2.508575


In [16]:
df.tail(10)

,District,Year,Avg_Rainfall,Avg_Temperature,Crop_Yield,Irrigation_Area,Rain_Next_1,Rain_Next_24,Rain_Next_7d
1310,PK726,2015,8.453675,20.559059,3006.106610,119.558635,-22.427746,-1.569843,-1.648335
1311,PK726,2016,6.903961,21.414617,2985.375839,120.281879,21.015043,-7.533452,-7.910124
1312,PK726,2017,9.817506,20.417397,2969.108481,118.157791,-17.467158,-5.456601,-5.729431
1313,PK726,2018,3.318286,20.417397,2958.949495,119.383838,-31.278680,-2.404250,-2.524463
1314,PK726,2019,10.485631,20.417397,2959.044304,117.308017,-32.830786,-2.649555,-2.782033
1315,PK726,2020,12.421911,20.417397,2975.688525,118.704918,-36.210754,-2.966385,-3.114705
1316,PK726,2021,8.111194,20.417397,2973.113131,118.704918,-43.250654,-4.410348,-4.630865
1317,PK726,2022,13.480117,20.417397,2973.113131,118.704918,-82.780727,-2.726167,-2.862475
1318,PK726,2023,8.965433,20.417397,2973.113131,118.704918,-46.213859,-4.855047,-5.097800
1319,PK726,2024,7.396167,20.417397,2973.113131,118.704918,-83.040620,-2.683296,-2.817461


In [17]:
#fix negative values by clamping them to zero
df["Rain_Next_1"]  = df["Rain_Next_1"].clip(lower=0)
df["Rain_Next_24"] = df["Rain_Next_24"].clip(lower=0)
df["Rain_Next_7d"] = df["Rain_Next_7d"].clip(lower=0)

print("negative rainfall values fixed.")

negative rainfall values fixed.


In [18]:
df[["Rain_Next_1","Rain_Next_24","Rain_Next_7d"]].describe()

,Rain_Next_1,Rain_Next_24,Rain_Next_7d
count,1260.000000,1260.000000,1260.000000
mean,13.194416,2.122309,2.228424
std,35.360942,6.535777,6.862566
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,11.662720,2.397518,2.517394
max,314.945747,60.178833,63.187774


In [19]:
df.isna().sum()

District            0
Year                0
Avg_Rainfall        0
Avg_Temperature     0
Crop_Yield          0
Irrigation_Area     0
Rain_Next_1        60
Rain_Next_24       60
Rain_Next_7d       60
dtype: int64

In [20]:
df.shape

(1320, 9)

In [21]:
df_clean = df.dropna(subset=["Rain_Next_1", "Rain_Next_24", "Rain_Next_7d"])

In [23]:
df_clean.shape

(1260, 9)

In [24]:
numeric_cols = ["Avg_Rainfall", "Avg_Temperature", 
                "Crop_Yield", "Irrigation_Area",
                "Rain_Next_1", "Rain_Next_24", "Rain_Next_7d"]

df_clean[numeric_cols] = df_clean[numeric_cols].apply(pd.to_numeric, errors="coerce")

C:\Users\ibrah\AppData\Local\Temp\ipykernel_31956\3634478459.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[numeric_cols] = df_clean[numeric_cols].apply(pd.to_numeric, errors="coerce")


In [25]:
df_cat = df_clean.copy()
save_path_cat = r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\processed\final_catboost_dataset.csv"

df_cat.to_csv(save_path_cat, index=False)
print("saved CatBoost dataset →", save_path_cat)

saved CatBoost dataset → C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\processed\final_catboost_dataset.csv


In [27]:
df_cat.head(10)

,District,Year,Avg_Rainfall,Avg_Temperature,Crop_Yield,Irrigation_Area,Rain_Next_1,Rain_Next_24,Rain_Next_7d
2,PK203,1983,3.331681,19.611718,2973.113131,118.704918,10.591415,2.269813,2.383304
3,PK203,1984,2.419928,19.920437,2973.113131,118.704918,11.673900,2.423514,2.544689
4,PK203,1985,2.637431,20.242908,2973.113131,118.704918,11.465340,2.390116,2.509621
5,PK203,1986,2.657394,19.578992,2973.113131,118.704918,11.417544,2.382497,2.501622
6,PK203,1987,2.810428,20.427150,2973.113131,118.704918,11.513817,2.397244,2.517106
7,PK203,1988,3.055322,20.858067,2973.113131,118.704918,11.346742,2.370924,2.489470
8,PK203,1989,3.025525,19.541077,2973.113131,118.704918,11.504424,2.396040,2.515842
9,PK203,1990,3.990772,20.353342,2973.113131,118.704918,11.451955,2.389119,2.508575
10,PK203,1991,3.493017,19.842869,2973.113131,118.704918,11.253794,2.358220,2.476131
11,PK203,1992,3.450828,19.705822,2973.113131,118.704918,11.263318,2.358234,2.476146


In [28]:
from sklearn.preprocessing import LabelEncoder

df_ml = df_clean.copy()

#encode district
le = LabelEncoder()
df_ml["District_Code"] = le.fit_transform(df_ml["District"])

#drop the string version
df_ml = df_ml.drop(columns=["District"])

save_path_ml = r"C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\processed\final_ml_dataset.csv"

df_ml.to_csv(save_path_ml, index=False)
print("saved ML dataset →", save_path_ml)


saved ML dataset → C:\Ibrahim\Personal\University Stuff\Machine Learning\Project\ML Irrigation Project\data\processed\final_ml_dataset.csv


In [30]:
df_ml.tail(10)

,Year,Avg_Rainfall,Avg_Temperature,Crop_Yield,Irrigation_Area,Rain_Next_1,Rain_Next_24,Rain_Next_7d,District_Code
1310,2015,8.453675,20.559059,3006.106610,119.558635,0.000000,0.0,0.0,29
1311,2016,6.903961,21.414617,2985.375839,120.281879,21.015043,0.0,0.0,29
1312,2017,9.817506,20.417397,2969.108481,118.157791,0.000000,0.0,0.0,29
1313,2018,3.318286,20.417397,2958.949495,119.383838,0.000000,0.0,0.0,29
1314,2019,10.485631,20.417397,2959.044304,117.308017,0.000000,0.0,0.0,29
1315,2020,12.421911,20.417397,2975.688525,118.704918,0.000000,0.0,0.0,29
1316,2021,8.111194,20.417397,2973.113131,118.704918,0.000000,0.0,0.0,29
1317,2022,13.480117,20.417397,2973.113131,118.704918,0.000000,0.0,0.0,29
1318,2023,8.965433,20.417397,2973.113131,118.704918,0.000000,0.0,0.0,29
1319,2024,7.396167,20.417397,2973.113131,118.704918,0.000000,0.0,0.0,29
